In [1]:
# 提前导包
import numpy as np
import torch
from transformers import BertConfig,BertModel,BertTokenizer
from transformers import BertForMaskedLM,BertForNextSentencePrediction
from transformers import BertModel

model_name = 'bert-base-chinese'
MODEL_PATH = r'C:\Users\cccql\.cache\huggingface\hub\models--bert-base-chinese\snapshots\38fda776740d17609554e879e3ac7b9837bdb5ee'

tokenizer = BertTokenizer.from_pretrained(MODEL_PATH)
config = BertConfig.from_pretrained(MODEL_PATH)
config.output_hidden_states = True
config.output_attentions = True
model = BertModel.from_pretrained(MODEL_PATH,config=config)

model


d:\Awork\Anaconda3\envs\pytorch113\lib\site-packages\tqdm\auto.py:22: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
Some weights of the model checkpoint at C:\Users\cccql\.cache\huggingface\hub\models--bert-base-chinese\snapshots\38fda776740d17609554e879e3ac7b9837bdb5ee were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if

BertModel(
  (embeddings): BertEmbeddings(
    (word_embeddings): Embedding(21128, 768, padding_idx=0)
    (position_embeddings): Embedding(512, 768)
    (token_type_embeddings): Embedding(2, 768)
    (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (encoder): BertEncoder(
    (layer): ModuleList(
      (0): BertLayer(
        (attention): BertAttention(
          (self): BertSelfAttention(
            (query): Linear(in_features=768, out_features=768, bias=True)
            (key): Linear(in_features=768, out_features=768, bias=True)
            (value): Linear(in_features=768, out_features=768, bias=True)
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (output): BertSelfOutput(
            (dense): Linear(in_features=768, out_features=768, bias=True)
            (LayerNorm): LayerNorm((768,), eps=1e-12, elementwise_affine=True)
            (dropout): Dropout(p=0.1, inplace=False)
          

In [10]:
'''
- encode仅返回input_ids
- encode_plus返回所有编码信息
    - ‘input_ids’：是单词在词典中的编码
    - ‘token_type_ids’：区分两个句子的编码（上句全为0，下句全为1）
    - ‘attention_mask’：指定对哪些词进行self-Attention操作
'''
print(tokenizer([['我不喜欢这个世界。','我也是！'],['asdd','xcvz    ']]))
print(tokenizer.encode('我不喜欢这个世界。'))
print(tokenizer.encode_plus('我不喜欢这个世界。'))
a = tokenizer.encode_plus('我不喜欢这个世界。')
a.input_ids

{'input_ids': [[101, 2769, 679, 1599, 3614, 6821, 702, 686, 4518, 511, 102, 2769, 738, 3221, 8013, 102], [101, 8975, 9879, 102, 166, 8177, 8225, 8253, 102]], 'token_type_ids': [[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 1, 1, 1, 1, 1], [0, 0, 0, 0, 1, 1, 1, 1, 1]], 'attention_mask': [[1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1], [1, 1, 1, 1, 1, 1, 1, 1, 1]]}
[101, 2769, 679, 1599, 3614, 6821, 702, 686, 4518, 511, 102]
{'input_ids': [101, 2769, 679, 1599, 3614, 6821, 702, 686, 4518, 511, 102], 'token_type_ids': [0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0], 'attention_mask': [1, 1, 1, 1, 1, 1, 1, 1, 1, 1, 1]}


[101, 2769, 679, 1599, 3614, 6821, 702, 686, 4518, 511, 102]

In [11]:
# 将input_ids转化回token

print(tokenizer.convert_ids_to_tokens(tokenizer.encode('我不喜欢这个世界。')))

['[CLS]', '我', '不', '喜', '欢', '这', '个', '世', '界', '。', '[SEP]']


In [12]:
# 将分词输入模型，得到编码
torch.tensor([a.input_ids]).shape

torch.Size([1, 11])

In [5]:
torch.tensor(a.input_ids).shape

torch.Size([11])

In [14]:
tokens_tensor = torch.tensor([a.input_ids])
sagment_tensors = torch.tensor([a.token_type_ids])
sagment_tensors

tensor([[0, 0, 0, 0, 0, 0, 0, 0, 0, 0, 0]])

In [25]:
model.eval()

with torch.no_grad():
    output = model(tokens_tensor,token_type_ids = sagment_tensors)
    encoded_hidden = output
encoded_hidden[3]

(tensor([[[[5.8265e-04, 9.1112e-04, 2.0561e-03,  ..., 1.0617e-03,
            2.1599e-03, 9.8682e-01],
           [4.0710e-02, 1.4153e-01, 4.1808e-02,  ..., 1.2148e-01,
            8.3455e-02, 9.3920e-03],
           [1.6007e-01, 7.0425e-02, 5.5830e-02,  ..., 1.0093e-01,
            8.9538e-02, 1.0958e-02],
           ...,
           [1.5883e-01, 1.2269e-01, 1.5134e-02,  ..., 8.6266e-02,
            1.5416e-01, 1.2672e-02],
           [3.0713e-01, 4.7508e-02, 6.3705e-02,  ..., 4.1128e-02,
            2.1358e-01, 2.9280e-02],
           [8.6614e-01, 9.8779e-03, 1.4715e-02,  ..., 8.4041e-03,
            1.4768e-02, 8.6476e-03]],
 
          [[1.2658e-02, 3.2177e-02, 1.6318e-02,  ..., 3.3708e-02,
            2.6675e-02, 8.4058e-01],
           [4.2186e-01, 8.2696e-02, 2.0308e-01,  ..., 2.7415e-02,
            1.4561e-02, 9.0180e-02],
           [2.3303e-01, 3.6908e-01, 1.5277e-02,  ..., 4.4839e-03,
            1.5576e-02, 6.0964e-03],
           ...,
           [1.2024e-02, 8.2655e-04, 8.

In [49]:
# 1.遮蔽语言模型

# 任务一：遮蔽语言模型
# BERT 在预训练中引入[CLS]和[SEP]标记句子和结尾
samples = ['[CLS]中国的首都是哪里？[SEP]北京是[MASK]国的首都。[SEP]']  # 准备输入模型的语句

# tokenized_text = [tokenizer.tokenize(i) for i in samples]
# input_text = torch.LongTensor(tokenized_text)


tokenized_text = [tokenizer.tokenize(i) for i in samples]                 #将句子分割成一个个token，即一个个汉字和分隔符
input_ids = [tokenizer.convert_tokens_to_ids(i) for i in tokenized_text]  #把每个token转换成对应的索引
input_ids = torch.LongTensor(input_ids)

model = BertForMaskedLM.from_pretrained(MODEL_PATH)
model.eval()
outputs = model(input_ids)
print(outputs[0])

pre_score = outputs[0]
sample = pre_score[0].detach().numpy()
pred = np.argmax(sample,axis=1)
print(tokenizer.convert_ids_to_tokens(pred)[14])



Some weights of the model checkpoint at C:\Users\cccql\.cache\huggingface\hub\models--bert-base-chinese\snapshots\38fda776740d17609554e879e3ac7b9837bdb5ee were not used when initializing BertForMaskedLM: ['cls.seq_relationship.bias', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertForMaskedLM from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForMaskedLM from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[[ -7.9952,  -7.9039,  -7.9825,  ...,  -6.9982,  -7.0039,  -7.0533],
         [ -7.3028,  -7.6208,  -7.4104,  ...,  -5.9638,  -6.7491,  -3.6037],
         [-18.8863, -18.8394, -18.4806,  ..., -14.1091,  -9.2716, -10.0511],
         ...,
         [-15.1031, -15.8225, -14.4563,  ..., -10.7002, -10.1100,  -6.4397],
         [-11.3589, -11.2920, -10.8067,  ...,  -8.5306,  -5.9084,  -5.8770],
         [ -9.2334,  -9.1561,  -9.3433,  ...,  -6.9585,  -6.7155,  -6.3854]]],
       grad_fn=<ViewBackward0>)
中


In [60]:
# 2.句子预测任务

sen_code1 = tokenizer.encode_plus('天气怎么样','今天的天气很好啊')
sen_code2 = tokenizer.encode_plus('飞机在天上','我在吃披萨和牛排')

tokens_tensor = torch.tensor([sen_code1['input_ids'],sen_code2['input_ids']])
model = BertForNextSentencePrediction.from_pretrained(MODEL_PATH)   
model.eval()
outputs = model(tokens_tensor)
seq_relationship_scores = outputs[0]
print(seq_relationship_scores)
sample = seq_relationship_scores.detach().numpy()
pred = np.argmax(sample,axis = 1)
print(pred)


Some weights of the model checkpoint at C:\Users\cccql\.cache\huggingface\hub\models--bert-base-chinese\snapshots\38fda776740d17609554e879e3ac7b9837bdb5ee were not used when initializing BertForNextSentencePrediction: ['cls.predictions.bias', 'cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight']
- This IS expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForNextSentencePrediction from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).


tensor([[2.9951, 1.2042],
        [3.1683, 0.8106]], grad_fn=<AddmmBackward0>)
[0 0]


In [ ]:
# 前面三行代码等价于：
samples = ["[CLS]今天天气怎么样[SEP]今天天气很好[SEP]", "[CLS]小明今年几岁了[SEP]我不喜欢学习[SEP]"]
tokenizer = BertTokenizer.from_pretrained(model_name)
tokenized_text = [tokenizer.tokenize(i) for i in samples]                  
input_ids = [tokenizer.convert_tokens_to_ids(i) for i in tokenized_text]   

tokens_tensor = torch.LongTensor(input_ids)

In [69]:
from transformers import BertForQuestionAnswering
model_name = 'bert-base-chinese'

 # 通过词典导入分词器
tokenizer = BertTokenizer.from_pretrained(model_name)
 # 导入配置文件
model_config = BertConfig.from_pretrained(model_name)
  # 最终有两个输出，初始位置和结束位置
model_config.num_labels = 2
 
# 根据bert的model_config新建BertForQuestionAnswering
model = BertForQuestionAnswering(model_config)
model.eval()

question, text = "里昂是谁", "里昂是一个杀手"
 
sen_code = tokenizer.encode_plus(question,text)

tokens_tensor = torch.tensor([sen_code['input_ids']])
segments_tensor = torch.tensor([sen_code['token_type_ids']])

start_pos, end_pos = model(tokens_tensor, segments_tensor)
 # 进行逆编码，得到原始的token
all_tokens = tokenizer.convert_ids_to_tokens(sen_code['input_ids'])
print(all_tokens)               #['[CLS]', '里', '昂', '是', '谁', '[SEP]', '里', '昂', '是', '一', '个', '杀', '手', '[SEP]']

 # 对输出的答案进行解码的过程
answer = ' '.join(all_tokens[torch.argmax(start_pos,axis = 1) : torch.argmax(end_pos,axis = 1)+1])

# 每次执行的结果不一致，这里因为没有经过微调，所以效果不是很好，输出结果不佳，下面的输出是其中的一种。
print(answer)                    #一 个 杀 手 [SEP]

['[CLS]', '里', '昂', '是', '谁', '[SEP]', '里', '昂', '是', '一', '个', '杀', '手', '[SEP]']


TypeError: argmax(): argument 'input' (position 1) must be Tensor, not str